# 1. Imports

In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from geopy.distance import geodesic
import time
import ast
tqdm.pandas()
from datetime import datetime as dt
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")

# 2. Request data from data.gov.sg API
Link: https://data.gov.sg/dataset/resale-flat-prices

In [3]:
url = 'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=200000'
r = requests.get(url)
data = r.json()
# data

In [4]:
len(data['result']['records'])

149326

In [5]:
df = pd.DataFrame(data['result']['records'])
df.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block
0,ANG MO KIO,2 ROOM,Improved,44,ANG MO KIO AVE 10,232000,2017-01,61 years 04 months,1979,10 TO 12,1,406
1,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 4,250000,2017-01,60 years 07 months,1978,01 TO 03,2,108
2,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 5,262000,2017-01,62 years 05 months,1980,01 TO 03,3,602
3,ANG MO KIO,3 ROOM,New Generation,68,ANG MO KIO AVE 10,265000,2017-01,62 years 01 month,1980,04 TO 06,4,465
4,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 5,265000,2017-01,62 years 05 months,1980,01 TO 03,5,601


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149326 entries, 0 to 149325
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   town                 149326 non-null  object
 1   flat_type            149326 non-null  object
 2   flat_model           149326 non-null  object
 3   floor_area_sqm       149326 non-null  object
 4   street_name          149326 non-null  object
 5   resale_price         149326 non-null  object
 6   month                149326 non-null  object
 7   remaining_lease      149326 non-null  object
 8   lease_commence_date  149326 non-null  object
 9   storey_range         149326 non-null  object
 10  _id                  149326 non-null  int64 
 11  block                149326 non-null  object
dtypes: int64(1), object(11)
memory usage: 13.7+ MB


In [7]:
df['storey_range'].unique()

array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '37 TO 39',
       '49 TO 51', '28 TO 30', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45'], dtype=object)

In [8]:
# Data manipulation
float_cols = ['floor_area_sqm', 'resale_price']
df[float_cols] = df[float_cols].astype(float)

int_cols = ['lease_commence_date']
df[int_cols] = df[int_cols].astype(float)

def get_mths_remaining_lease(lease_str):
    split_lst = lease_str.split(' ')
    if len(split_lst) == 4:
        return int(split_lst[0]) * 12 + int(split_lst[2])
    else:
        return int(split_lst[0]) * 12
    
df['remaining_lease_mth'] = df['remaining_lease'].apply(get_mths_remaining_lease)
df['storey_range_lower'] = df['storey_range'].apply(lambda x: int(x.split(' ')[0]))

df.drop(['remaining_lease'], axis = 1, inplace = True)
df.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,block,remaining_lease_mth,storey_range_lower
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,406,736,10
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,108,727,1
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,602,749,1
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,465,745,4
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,601,749,1


In [9]:
df['address_str'] = df['block'] + " " + df['street_name']

# 3. Feature Engineering

In [10]:
# Get subset to test here first
# test_df = df[:100]
final_df = df.copy()

## 3.1. Coordinates

In [11]:
# Download lat long from onemap API
def get_coordinates(address):
    for i in range(10):
        try:
            req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
            resultsdict = eval(req.text)
            if len(resultsdict['results']) > 0:
                return (resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE'])
            else:
                return np.nan
            break
        except:
            time.sleep(10)

final_df['lat_lon'] = final_df['address_str'].progress_apply(get_coordinates)

100%|██████████| 149326/149326 [4:27:58<00:00,  9.29it/s]  


In [12]:
# Save checkpoint 1 (4h 30mins for 149,326 rows)
final_df.to_csv('checkpoint_1.csv', index=False)


In [3]:
# Read in checkpoint 1
final_df = pd.read_csv('checkpoint_1.csv')
# df_read_in.info()
final_df['lat_lon'] = final_df['lat_lon'].apply(ast.literal_eval).apply(lambda x: (float(x[0]), float(x[1]))) # Change to tuple of floats

In [6]:
def get_coordinates(address):
    for i in range(10):
        try:
            req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
            resultsdict = eval(req.text)
            if len(resultsdict['results']) > 0:
                return (resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE'])
            else:
                return np.nan
            break
        except:
            time.sleep(10)

## 3.2. MRT Stations
List of MRT stations obtained on 3/3/2023 from https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations

In [7]:
mrt_df = pd.read_excel('MRT_List.xlsx')
mrt_df['MRT Stations'] = mrt_df['MRT Stations'] + ' MRT Station'
mrt_df['MRT_stn_lat_long'] = mrt_df['MRT Stations'].progress_apply(get_coordinates) # Download lat long from onemap API
mrt_df = mrt_df.dropna()
mrt_df = mrt_df.reset_index(drop = True)
mrt_df

100%|██████████| 146/146 [00:12<00:00, 11.44it/s]


,MRT Stations,MRT_stn_lat_long
0,Admiralty MRT Station,"(1.44058856161847, 103.800990519771)"
1,Aljunied MRT Station,"(1.3164326118157, 103.882906044385)"
2,Ang Mo Kio MRT Station,"(1.36942855699191, 103.849455226442)"
3,Bartley MRT Station,"(1.34250117805245, 103.880177899184)"
4,Bayfront MRT Station,"(1.28187378879209, 103.859079764874)"
...,...,...
138,Woodleigh MRT Station,"(1.33919004519388, 103.87081830915)"
139,Xilin MRT Station,"(1.32890694938858, 103.964902569595)"
140,Yew Tee MRT Station,"(1.39747594171731, 103.747418249132)"
141,Yio Chu Kang MRT Station,"(1.38168259989517, 103.844991053696)"


In [8]:
def get_mrt_shortest_and_1km_radius(place_coord):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        dist_df = mrt_df['MRT_stn_lat_long'].apply(lambda x: geodesic(place_coord, x).meters)
        min_dist = dist_df.min()
        mrt_stn = mrt_df.loc[dist_df.idxmin(),'MRT Stations']
        num_within_1km_radius = sum(dist_df <= 1000)
        return (min_dist, mrt_stn, num_within_1km_radius)

output = final_df['lat_lon'].progress_apply(get_mrt_shortest_and_1km_radius)
final_df['closest_mrt_dist'] = output.apply(lambda x: x[0])
final_df['closest_mrt'] = output.apply(lambda x: x[1])
final_df['num_mrt_1km_radius'] = output.apply(lambda x: x[2])
final_df.head()

100%|██████████| 149326/149326 [45:57<00:00, 54.15it/s] 


,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,block,remaining_lease_mth,storey_range_lower,address_str,lat_lon,closest_mrt_dist,closest_mrt,num_mrt_1km_radius
0,0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,406,736,10,406 ANG MO KIO AVE 10,"(1.36200453938712, 103.853879910407)",957.269782,Ang Mo Kio MRT Station,1
1,1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,108,727,1,108 ANG MO KIO AVE 4,"(1.37094273993861, 103.837974822369)",166.828293,Mayflower MRT Station,2
2,2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,602,749,1,602 ANG MO KIO AVE 5,"(1.38070883044887, 103.835368226602)",532.154773,Lentor MRT Station,1
3,3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,465,745,4,465 ANG MO KIO AVE 10,"(1.3662010408294, 103.857200967235)",932.964388,Ang Mo Kio MRT Station,1
4,4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,601,749,1,601 ANG MO KIO AVE 5,"(1.38104134784496, 103.835131744823)",498.418981,Lentor MRT Station,1


## 3.3. Schools

In [9]:
school_df_dic = pd.read_excel('Schools.xlsx', sheet_name = None)
school_df = pd.DataFrame()
for sch_type, df in school_df_dic.items():
    sub_df = pd.DataFrame({'Type': sch_type, 'School': df.iloc[:,0]})
    school_df = pd.concat([school_df, sub_df], ignore_index = True, axis = 0)
    
school_df['lat_long'] = school_df['School'].progress_apply(get_coordinates)
school_df = school_df.dropna()
school_df

100%|██████████| 373/373 [00:41<00:00,  8.89it/s]


,Type,School,lat_long
0,Pri,Admiralty Primary School,"(1.4426347903311, 103.800040119743)"
1,Pri,Ahmad Ibrahim Primary School,"(1.43315271543517, 103.832942401086)"
2,Pri,Ai Tong School,"(1.3605834338904, 103.833020333986)"
3,Pri,Alexandra Primary School,"(1.29133439161334, 103.824424680531)"
4,Pri,Anchor Green Primary School,"(1.39036998654612, 103.887165375933)"
...,...,...,...
368,Uni,Nanyang Technological University,"(1.35154257621121, 103.687214243669)"
369,Uni,Singapore Management University,"(1.29509045190873, 103.850566200283)"
370,Uni,Singapore University of Technology and Design,"(1.34078410232498, 103.962542022484)"
371,Uni,Singapore Institute of Technology,"(1.34342966240204, 103.932354333699)"


In [10]:
sch_df_dic = {}
for sch_type in school_df['Type'].unique():
    sub_df = school_df[school_df['Type'] == sch_type]
    sub_df = sub_df.reset_index(drop = True)
    sch_df_dic[sch_type] = sub_df

In [11]:
def get_sch_shortest_and_1km_radius(place_coord, sch_type):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        sub_df = sch_df_dic[sch_type]
        dist_df = sub_df['lat_long'].apply(lambda x: geodesic(place_coord, x).meters)
        min_dist = dist_df.min()
        nearest_sch = sub_df.loc[dist_df.idxmin(), 'School']
        num_within_1km_radius = sum(dist_df <= 1000)
        return (min_dist, nearest_sch, num_within_1km_radius)

for sch_type in sch_df_dic.keys():
    output = final_df['lat_lon'].progress_apply(lambda x: get_sch_shortest_and_1km_radius(x, sch_type))
    final_df[f'closest_{sch_type}_sch_dist'] = output.apply(lambda x: x[0])
    final_df[f'closest_{sch_type}_sch'] = output.apply(lambda x: x[1])
    final_df[f'num_{sch_type}_sch_1km_radius'] = output.apply(lambda x: x[2])
final_df.head()

100%|██████████| 149326/149326 [02:24<00:00, 1032.31it/s]


,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,...,num_Sec_sch_1km_radius,closest_JC_sch_dist,closest_JC_sch,num_JC_sch_1km_radius,closest_MI_sch_dist,closest_MI_sch,num_MI_sch_1km_radius,closest_Uni_sch_dist,closest_Uni_sch,num_Uni_sch_1km_radius
0,0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,...,1,1401.612589,Eunoia Junior College,0,1779.216231,ITE College Central,0,7408.201434,Singapore Management University,0
1,1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,...,5,1041.037313,Eunoia Junior College,0,1726.056136,Nanyang Polytechnic,0,8371.125772,Singapore University of Social Sciences,0
2,2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,...,3,1172.923989,Anderson Serangoon Junior College,0,1599.601569,Nanyang Polytechnic,0,8802.805943,Singapore University of Social Sciences,0
3,3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,...,1,1827.233537,Eunoia Junior College,0,1295.710648,ITE College Central,0,7897.637286,Singapore Management University,0
4,4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,...,3,1205.413198,Anderson Serangoon Junior College,0,1625.299098,Nanyang Polytechnic,0,8807.296192,Singapore University of Social Sciences,0


In [13]:
# Save checkpoint 2
final_df.to_csv('checkpoint_2.csv', index=False)

In [8]:
# Read in checkpoint 2
final_df = pd.read_csv('checkpoint_2.csv')
# df_read_in.info()
final_df['lat_lon'] = final_df['lat_lon'].apply(ast.literal_eval).apply(lambda x: (float(x[0]), float(x[1]))) # Change to tuple of floats

## 3.4. Malls

In [7]:
malls_df_dic = pd.read_excel('Malls.xlsx', sheet_name = None)

malls_df = pd.DataFrame()
for mall_type, df in malls_df_dic.items():
    sub_df = df.iloc[:,0]
    malls_df = pd.concat([malls_df, sub_df], ignore_index = True, axis = 0)
    
malls_df.columns = ['Malls']

print(malls_df.shape)
malls_df['lat_long'] = malls_df['Malls'].progress_apply(get_coordinates) # progress_apply
malls_df = malls_df.dropna()
malls_df = malls_df.reset_index(drop = True)

malls_df

(167, 1)


100%|█████████████████████████████████████████| 167/167 [00:13<00:00, 12.70it/s]


,Malls,lat_long
0,100 AM,"(1.27468281482263, 103.843488359469)"
1,313@Somerset,"(1.30101436404056, 103.838360664485)"
2,Aperia,"(1.3097112065077, 103.864326436447)"
3,Balestier Hill Shopping Centre,"(1.32559594839311, 103.842571612968)"
4,Bugis Cube,"(1.2981408343975, 103.855635339249)"
...,...,...
155,Gek Poh Shopping Centre,"(1.34874357136408, 103.697732091001)"
156,Rochester Mall,"(1.30540765569962, 103.788446680148)"
157,Taman Jurong Shopping Centre,"(1.33484487471259, 103.720462024278)"
158,West Coast Plaza,"(1.30369748971099, 103.766131294678)"


In [9]:
def get_mall_shortest_and_1km_radius(place_coord):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        dist_df = malls_df['lat_long'].apply(lambda x: geodesic(place_coord, x).meters)
        min_dist = dist_df.min()
        mall = malls_df.loc[dist_df.idxmin(), 'Malls']
        num_within_1km_radius = sum(dist_df <= 1000)
        return (min_dist, mall, num_within_1km_radius)

output = final_df['lat_lon'].progress_apply(get_mall_shortest_and_1km_radius)
final_df['closest_mall_dist'] = output.apply(lambda x: x[0])
final_df['closest_mall'] = output.apply(lambda x: x[1])
final_df['num_mall_1km_radius'] = output.apply(lambda x: x[2])
final_df.head()

100%|███████████████████████████████████| 149326/149326 [28:21<00:00, 87.76it/s]


,Unnamed: 0.1,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,...,num_JC_sch_1km_radius,closest_MI_sch_dist,closest_MI_sch,num_MI_sch_1km_radius,closest_Uni_sch_dist,closest_Uni_sch,num_Uni_sch_1km_radius,closest_mall_dist,closest_mall,num_mall_1km_radius
0,0,0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,...,0,1779.216231,ITE College Central,0,7408.201434,Singapore Management University,0,1013.992058,AMK Hub,0
1,1,1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,...,0,1726.056136,Nanyang Polytechnic,0,8371.125772,Singapore University of Social Sciences,0,894.265975,Broadway Plaza,1
2,2,2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,...,0,1599.601569,Nanyang Polytechnic,0,8802.805943,Singapore University of Social Sciences,0,1525.573038,Broadway Plaza,0
3,3,3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,...,0,1295.710648,ITE College Central,0,7897.637286,Singapore Management University,0,893.795634,myVillage At Serangoon Garden,1
4,4,4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,...,0,1625.299098,Nanyang Polytechnic,0,8807.296192,Singapore University of Social Sciences,0,1569.306143,Broadway Plaza,0


## 3.5. Parks

In [10]:
parks_df = pd.read_excel('Parks.xlsx')
parks_df = pd.DataFrame({'Parks': parks_df.iloc[:,0]})
parks_df['lat_long'] = parks_df['Parks'].progress_apply(get_coordinates)
parks_df = parks_df.dropna()
parks_df = parks_df.reset_index(drop = True)
parks_df

100%|███████████████████████████████████████████| 73/73 [00:05<00:00, 12.51it/s]


,Parks,lat_long
0,Admiralty Park,"(1.45399059016631, 103.795875774662)"
1,Ang Mo Kio Town Garden East,"(1.37069218596975, 103.850477038504)"
2,Ang Mo Kio Town Garden West,"(1.37416758787361, 103.842948749695)"
3,Bedok Town Park,"(1.33439933330273, 103.922348727352)"
4,Bishan-Ang Mo Kio Park,"(1.36517069591469, 103.83629822663)"
...,...,...
65,Yishun Neighbourhood Park,"(1.43774180535158, 103.835167452882)"
66,Yishun Park,"(1.44144025471558, 103.835602638346)"
67,Yishun Pond Park,"(1.42782773147436, 103.840061845933)"
68,Youth Olympic Park,"(1.28895335576015, 103.860431957375)"


In [11]:
def get_park_shortest_and_1km_radius(place_coord):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        dist_df = parks_df['lat_long'].apply(lambda x: geodesic(place_coord, x).meters)
        min_dist = dist_df.min()
        park = parks_df.loc[dist_df.idxmin(), 'Parks']
        num_within_1km_radius = sum(dist_df <= 1000)
        return (min_dist, park, num_within_1km_radius)

output = final_df['lat_lon'].progress_apply(get_park_shortest_and_1km_radius)
final_df['closest_park_dist'] = output.apply(lambda x: x[0])
final_df['closest_park'] = output.apply(lambda x: x[1])
final_df['num_park_1km_radius'] = output.apply(lambda x: x[2])
final_df.head()

100%|██████████████████████████████████| 149326/149326 [12:50<00:00, 193.91it/s]


,Unnamed: 0.1,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,...,num_MI_sch_1km_radius,closest_Uni_sch_dist,closest_Uni_sch,num_Uni_sch_1km_radius,closest_mall_dist,closest_mall,num_mall_1km_radius,closest_park_dist,closest_park,num_park_1km_radius
0,0,0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,...,0,7408.201434,Singapore Management University,0,1013.992058,AMK Hub,0,1032.586022,Ang Mo Kio Town Garden East,0
1,1,1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,...,0,8371.125772,Singapore University of Social Sciences,0,894.265975,Broadway Plaza,1,658.451197,Ang Mo Kio Town Garden West,2
2,2,2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,...,0,8802.805943,Singapore University of Social Sciences,0,1525.573038,Broadway Plaza,0,1111.238041,Ang Mo Kio Town Garden West,0
3,3,3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,...,0,7897.637286,Singapore Management University,0,893.795634,myVillage At Serangoon Garden,1,898.087299,Ang Mo Kio Town Garden East,1
4,4,4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,...,0,8807.296192,Singapore University of Social Sciences,0,1569.306143,Broadway Plaza,0,1155.199842,Ang Mo Kio Town Garden West,0


# Save Dataset into checkpoint3.csv

In [12]:
# Save checkpoint 3
final_df.to_csv('checkpoint_3.csv', index=False)

In [6]:
# Read in checkpoint 3
final_df = pd.read_csv('checkpoint_3.csv')
# df_read_in.info()
final_df['lat_lon'] = final_df['lat_lon'].apply(ast.literal_eval).apply(lambda x: (float(x[0]), float(x[1]))) # Change to tuple of floats

In [7]:
final_df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'town', 'flat_type',
       'flat_model', 'floor_area_sqm', 'street_name', 'resale_price', 'month',
       'lease_commence_date', 'storey_range', '_id', 'block',
       'remaining_lease_mth', 'storey_range_lower', 'address_str', 'lat_lon',
       'closest_mrt_dist', 'closest_mrt', 'num_mrt_1km_radius',
       'closest_Pri_sch_dist', 'closest_Pri_sch', 'num_Pri_sch_1km_radius',
       'closest_Sec_sch_dist', 'closest_Sec_sch', 'num_Sec_sch_1km_radius',
       'closest_JC_sch_dist', 'closest_JC_sch', 'num_JC_sch_1km_radius',
       'closest_MI_sch_dist', 'closest_MI_sch', 'num_MI_sch_1km_radius',
       'closest_Uni_sch_dist', 'closest_Uni_sch', 'num_Uni_sch_1km_radius',
       'closest_mall_dist', 'closest_mall', 'num_mall_1km_radius',
       'closest_park_dist', 'closest_park', 'num_park_1km_radius'],
      dtype='object')

## 3.6. Price per sqm

In [8]:
final_df['price_per_sqm'] = final_df['resale_price'] / final_df['floor_area_sqm']
final_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,...,closest_Uni_sch_dist,closest_Uni_sch,num_Uni_sch_1km_radius,closest_mall_dist,closest_mall,num_mall_1km_radius,closest_park_dist,closest_park,num_park_1km_radius,price_per_sqm
0,0,0,0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,...,7408.201434,Singapore Management University,0,1013.992058,AMK Hub,0,1032.586022,Ang Mo Kio Town Garden East,0,5272.727273
1,1,1,1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,...,8371.125772,Singapore University of Social Sciences,0,894.265975,Broadway Plaza,1,658.451197,Ang Mo Kio Town Garden West,2,3731.343284
2,2,2,2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,...,8802.805943,Singapore University of Social Sciences,0,1525.573038,Broadway Plaza,0,1111.238041,Ang Mo Kio Town Garden West,0,3910.447761
3,3,3,3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,...,7897.637286,Singapore Management University,0,893.795634,myVillage At Serangoon Garden,1,898.087299,Ang Mo Kio Town Garden East,1,3897.058824
4,4,4,4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,...,8807.296192,Singapore University of Social Sciences,0,1569.306143,Broadway Plaza,0,1155.199842,Ang Mo Kio Town Garden West,0,3955.223881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149321,149321,149321,149321,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN CTRL,800000.0,2023-03,...,13379.711491,Singapore University of Social Sciences,0,432.877723,Junction Nine,2,279.490328,Yishun Pond Park,1,5479.452055
149322,149322,149322,149322,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN RING RD,800000.0,2023-03,...,13232.047301,Singapore University of Social Sciences,0,931.337725,Wisteria Mall,2,661.649827,Yishun Pond Park,1,5479.452055
149323,149323,149323,149323,YISHUN,EXECUTIVE,Apartment,142.0,YISHUN RING RD,770000.0,2023-03,...,13428.149563,Singapore Institute of Technology,0,739.745345,Junction Nine,1,825.343534,Yishun Pond Park,1,5422.535211
149324,149324,149324,149324,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN RING RD,818000.0,2023-03,...,11979.803115,Singapore University of Social Sciences,0,965.509600,Northpoint City,1,976.514291,Sembawang Park,1,5602.739726


## 3.7. Town Population

In [9]:
df_population = pd.read_excel('Population.xlsx')
df_dwellings = pd.read_excel('Residential Dwellings.xlsx', header=2, skipfooter=8)

In [10]:
final_df['town'] = final_df['town'].replace({'KALLANG/WHAMPOA': 'KALLANG'})

In [11]:
towns = list(final_df['town'].unique())
towns

['ANG MO KIO',
 'BEDOK',
 'BISHAN',
 'BUKIT MERAH',
 'BUKIT BATOK',
 'CENTRAL AREA',
 'BUKIT PANJANG',
 'GEYLANG',
 'BUKIT TIMAH',
 'CHOA CHU KANG',
 'CLEMENTI',
 'PUNGGOL',
 'HOUGANG',
 'QUEENSTOWN',
 'JURONG EAST',
 'JURONG WEST',
 'KALLANG',
 'MARINE PARADE',
 'PASIR RIS',
 'SEMBAWANG',
 'SENGKANG',
 'SERANGOON',
 'TAMPINES',
 'TOA PAYOH',
 'WOODLANDS',
 'YISHUN']

In [12]:
df_dwellings['Planning Areas'].unique()

array(['Total', 'Ang Mo Kio', nan, 'Bedok', 'Bishan', 'Boon Lay',
       'Bukit Batok', 'Bukit Merah', 'Bukit Panjang', 'Bukit Timah',
       'Central Water Catchment', 'Changi', 'Changi Bay', 'Choa Chu Kang',
       'Clementi', 'Downtown Core', 'Geylang', 'Hougang', 'Jurong East',
       'Jurong West', 'Kallang', 'Lim Chu Kang', 'Mandai', 'Marina East',
       'Marina South', 'Marine Parade', 'Museum', 'Newton',
       'North-Eastern Islands', 'Novena', 'Orchard', 'Outram',
       'Pasir Ris', 'Paya Lebar', 'Pioneer', 'Punggol', 'Queenstown',
       'River Valley', 'Rochor', 'Seletar', 'Sembawang', 'Sengkang',
       'Serangoon', 'Simpang', 'Singapore River', 'Southern Islands',
       'Straits View', 'Sungei Kadut', 'Tampines', 'Tanglin', 'Tengah',
       'Toa Payoh', 'Tuas', 'Western Islands', 'Western Water Catchment',
       'Woodlands', 'Yishun'], dtype=object)

In [13]:
df_dwell = df_dwellings
df_dwell = df_dwell[(df_dwell['Subzones'] == 'Total')][['Planning Areas', 'Total*']]
df_dwell['Planning Areas'] = df_dwell['Planning Areas'].apply(lambda x: x.upper())
df_dwell = df_dwell[(df_dwell['Planning Areas']).isin(towns)]
df_dwell

,Planning Areas,Total*
1,ANG MO KIO,61730
14,BEDOK,105450
23,BISHAN,30400
32,BUKIT BATOK,58130
42,BUKIT MERAH,68220
60,BUKIT PANJANG,43930
68,BUKIT TIMAH,31600
85,CHOA CHU KANG,57570
92,CLEMENTI,38220
116,GEYLANG,49240


In [14]:
list(set(towns) - set(list(df_dwell['Planning Areas'].unique())))

['CENTRAL AREA']

In [15]:
df_merge = pd.merge(final_df, df_dwell, how='left', left_on='town', right_on='Planning Areas')
df_merge = df_merge.drop('Planning Areas', axis='columns')
df_merge.rename(columns={'Total*': 'num_of_dwellings_in_town'}, inplace=True)
df_merge

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,...,closest_Uni_sch,num_Uni_sch_1km_radius,closest_mall_dist,closest_mall,num_mall_1km_radius,closest_park_dist,closest_park,num_park_1km_radius,price_per_sqm,num_of_dwellings_in_town
0,0,0,0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,...,Singapore Management University,0,1013.992058,AMK Hub,0,1032.586022,Ang Mo Kio Town Garden East,0,5272.727273,61730
1,1,1,1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,...,Singapore University of Social Sciences,0,894.265975,Broadway Plaza,1,658.451197,Ang Mo Kio Town Garden West,2,3731.343284,61730
2,2,2,2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,...,Singapore University of Social Sciences,0,1525.573038,Broadway Plaza,0,1111.238041,Ang Mo Kio Town Garden West,0,3910.447761,61730
3,3,3,3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,...,Singapore Management University,0,893.795634,myVillage At Serangoon Garden,1,898.087299,Ang Mo Kio Town Garden East,1,3897.058824,61730
4,4,4,4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,...,Singapore University of Social Sciences,0,1569.306143,Broadway Plaza,0,1155.199842,Ang Mo Kio Town Garden West,0,3955.223881,61730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149321,149321,149321,149321,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN CTRL,800000.0,2023-03,...,Singapore University of Social Sciences,0,432.877723,Junction Nine,2,279.490328,Yishun Pond Park,1,5479.452055,75420
149322,149322,149322,149322,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN RING RD,800000.0,2023-03,...,Singapore University of Social Sciences,0,931.337725,Wisteria Mall,2,661.649827,Yishun Pond Park,1,5479.452055,75420
149323,149323,149323,149323,YISHUN,EXECUTIVE,Apartment,142.0,YISHUN RING RD,770000.0,2023-03,...,Singapore Institute of Technology,0,739.745345,Junction Nine,1,825.343534,Yishun Pond Park,1,5422.535211,75420
149324,149324,149324,149324,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN RING RD,818000.0,2023-03,...,Singapore University of Social Sciences,0,965.509600,Northpoint City,1,976.514291,Sembawang Park,1,5602.739726,75420


In [16]:
df_merge[df_merge['num_of_dwellings_in_town'].isna()]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,...,closest_Uni_sch,num_Uni_sch_1km_radius,closest_mall_dist,closest_mall,num_mall_1km_radius,closest_park_dist,closest_park,num_park_1km_radius,price_per_sqm,num_of_dwellings_in_town
187,187,187,187,CENTRAL AREA,4 ROOM,Type S1,95.0,CANTONMENT RD,903000.0,2017-01,...,Singapore Management University,0,140.516733,Tanjong Pagar Centre,3,574.360590,Tanjong Pagar Park,2,9505.263158,NaN
208,208,208,208,CENTRAL AREA,5 ROOM,Type S2,105.0,CANTONMENT RD,1108000.0,2017-01,...,Singapore Management University,0,267.363982,Tanjong Pagar Centre,4,564.157162,Tanjong Pagar Park,2,10552.380952,NaN
302,302,302,302,CENTRAL AREA,2 ROOM,Improved,55.0,JLN KUKOH,250000.0,2017-01,...,Singapore Management University,0,577.618043,People's Park Complex,4,349.613047,Pearl's Hill City Park,2,4545.454545,NaN
303,303,303,303,CENTRAL AREA,3 ROOM,Model A,84.0,ROWELL RD,360000.0,2017-01,...,Singapore Management University,0,486.177020,City Square Mall,7,1371.615574,Dhoby Ghaut Green,0,4285.714286,NaN
304,304,304,304,CENTRAL AREA,3 ROOM,Improved,68.0,WATERLOO ST,400000.0,2017-01,...,Singapore Management University,1,251.014914,Sunshine Plaza,15,649.793761,Dhoby Ghaut Green,4,5882.352941,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148153,148153,148153,148153,CENTRAL AREA,4 ROOM,Model A,109.0,ROWELL RD,655000.0,2023-03,...,Singapore Management University,0,432.287356,City Square Mall,7,1434.032760,Dhoby Ghaut Green,0,6009.174312,NaN
148154,148154,148154,148154,CENTRAL AREA,4 ROOM,Model A,102.0,VEERASAMY RD,660000.0,2023-03,...,Singapore Management University,0,373.802359,Sim Lim Square,8,1245.721068,Dhoby Ghaut Green,0,6470.588235,NaN
148155,148155,148155,148155,CENTRAL AREA,5 ROOM,Type S2,106.0,CANTONMENT RD,1260000.0,2023-03,...,Singapore Management University,0,220.259991,Tanjong Pagar Centre,4,527.214563,Tanjong Pagar Park,2,11886.792453,NaN
148156,148156,148156,148156,CENTRAL AREA,5 ROOM,Type S2,107.0,CANTONMENT RD,1380000.0,2023-03,...,Singapore Management University,0,178.357528,Tanjong Pagar Centre,3,574.846181,Tanjong Pagar Park,2,12897.196262,NaN


In [17]:
df_population['Name'] = df_population['Name'].apply(lambda x: x.upper())
df_population = df_population[df_population['Name'].isin(towns)]
df_population

,Name,Region,Area (km²),Population[7],Density (/km²)
13,JURONG EAST,West,17.83,76390,4400
14,JURONG WEST,West,14.69,258240,18000
15,BEDOK,East,21.69,278270,13000
16,BUKIT BATOK,West,11.13,164560,14000
17,BUKIT MERAH,Central,14.34,149160,11000
18,BUKIT PANJANG,West,8.99,137020,15000
19,BUKIT TIMAH,Central,17.53,79270,4400
21,GEYLANG,Central,9.64,109150,11400
22,KALLANG,Central,9.17,100520,11000
24,PASIR RIS,East,15.02,145990,9600


In [18]:
df_population = df_population.append({'Name': 'CENTRAL AREA', 'Region': 'Central'}, ignore_index = True)

In [19]:
df_merge = pd.merge(df_merge, df_population, how='left', left_on='town', right_on='Name')
df_merge = df_merge.drop('Name', axis='columns')
df_merge.rename(columns={'Population[7]': 'Population'}, inplace=True)
df_merge

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,...,num_mall_1km_radius,closest_park_dist,closest_park,num_park_1km_radius,price_per_sqm,num_of_dwellings_in_town,Region,Area (km²),Population,Density (/km²)
0,0,0,0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,...,0,1032.586022,Ang Mo Kio Town Garden East,0,5272.727273,61730,North-East,13.94,161000,13400
1,1,1,1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,...,1,658.451197,Ang Mo Kio Town Garden West,2,3731.343284,61730,North-East,13.94,161000,13400
2,2,2,2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,...,0,1111.238041,Ang Mo Kio Town Garden West,0,3910.447761,61730,North-East,13.94,161000,13400
3,3,3,3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,...,1,898.087299,Ang Mo Kio Town Garden East,1,3897.058824,61730,North-East,13.94,161000,13400
4,4,4,4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,...,0,1155.199842,Ang Mo Kio Town Garden West,0,3955.223881,61730,North-East,13.94,161000,13400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149321,149321,149321,149321,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN CTRL,800000.0,2023-03,...,2,279.490328,Yishun Pond Park,1,5479.452055,75420,North,21.24,222580,10100
149322,149322,149322,149322,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN RING RD,800000.0,2023-03,...,2,661.649827,Yishun Pond Park,1,5479.452055,75420,North,21.24,222580,10100
149323,149323,149323,149323,YISHUN,EXECUTIVE,Apartment,142.0,YISHUN RING RD,770000.0,2023-03,...,1,825.343534,Yishun Pond Park,1,5422.535211,75420,North,21.24,222580,10100
149324,149324,149324,149324,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN RING RD,818000.0,2023-03,...,1,976.514291,Sembawang Park,1,5602.739726,75420,North,21.24,222580,10100


# Save as checkpoint 4

In [20]:
# Save checkpoint 4
df_merge['resell_date'] = df_merge['month'].apply(lambda x: dt.strptime(x, '%Y-%m'))
df_merge.drop(columns = ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', '_id', 'month'], inplace = True)
df_merge.to_csv('checkpoint_4.csv', index=False)

In [15]:
# Read in checkpoint 4
final_df = pd.read_csv('checkpoint_4.csv')
final_df['lat_lon'] = final_df['lat_lon'].apply(ast.literal_eval).apply(lambda x: (float(x[0]), float(x[1]))) # Change to tuple of floats

## 3.8. Add in CPI Data

Source: https://www.rateinflation.com/consumer-price-index/singapore-historical-cpi/

Note: Annual CPI used, except 2023 where Jan 2023 CPI is used

In [16]:
cpi_df = pd.read_excel('CPI_Data.xlsx')
cpi_df

,Year,Annual CPI
0,2023,111.397
1,2022,108.370
2,2021,102.119
3,2020,99.818
4,2019,100.000
...,...,...
58,1965,25.513
59,1964,25.448
60,1963,25.033
61,1962,24.513


In [17]:
final_df['resale_year'] = final_df['resell_date'].apply(lambda x: dt.strptime(x, "%Y-%m-%d").year)
final_df = pd.merge(left=final_df, right=cpi_df, how='left', left_on='resale_year', right_on='Year')

In [19]:
final_df = final_df.drop(columns = ['resale_year', 'Year'])
final_df.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,lease_commence_date,storey_range,block,remaining_lease_mth,...,closest_park,num_park_1km_radius,price_per_sqm,num_of_dwellings_in_town,Region,Area (km²),Population,Density (/km²),resell_date,Annual CPI
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,1979.0,10 TO 12,406,736,...,Ang Mo Kio Town Garden East,0,5272.727273,61730.0,North-East,13.94,161000.0,13400.0,2017-01-01,99.004
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,1978.0,01 TO 03,108,727,...,Ang Mo Kio Town Garden West,2,3731.343284,61730.0,North-East,13.94,161000.0,13400.0,2017-01-01,99.004
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,1980.0,01 TO 03,602,749,...,Ang Mo Kio Town Garden West,0,3910.447761,61730.0,North-East,13.94,161000.0,13400.0,2017-01-01,99.004
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,1980.0,04 TO 06,465,745,...,Ang Mo Kio Town Garden East,1,3897.058824,61730.0,North-East,13.94,161000.0,13400.0,2017-01-01,99.004
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,1980.0,01 TO 03,601,749,...,Ang Mo Kio Town Garden West,0,3955.223881,61730.0,North-East,13.94,161000.0,13400.0,2017-01-01,99.004


## Save as checkpoint 5

In [21]:
# Save checkpoint 5
final_df.to_csv('checkpoint_5.csv', index=False)

In [22]:
# Read in checkpoint 5
final_df = pd.read_csv('checkpoint_5.csv')
final_df['lat_lon'] = final_df['lat_lon'].apply(ast.literal_eval).apply(lambda x: (float(x[0]), float(x[1]))) # Change to tuple of floats